In [ ]:
import torch
import sys
import gc

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, CODECONTESTS_COT_CONFIG, SMOL_LM_135M

In [ ]:
cfg = CODECONTESTS_COT_CONFIG
cfg.notebook_mode = True
cfg.model_id_or_path = SMOL_LM_135M
wrapper = TrainerWrapper(cfg)

In [ ]:
wrapper.init_model()

In [ ]:
wrapper.init_data_module()

In [ ]:
wrapper.tokenizer.name_or_path

In [ ]:
# Tokenizer dev
first_batch = wrapper.data_module.train_dataset[0]
# tokenized = wrapper.data_module.tokenize_conversation([first_batch["conversation"]])
for k in first_batch.keys():
    first_batch[k] = torch.as_tensor(first_batch[k], dtype=torch.long)
wrapper.data_module.visualize_sample(first_batch)

In [ ]:
# mock_labels = first_batch["labels"].clone().detach()
# mock_labels = torch.where(mock_labels == -100, wrapper.data_module.tokenizer.pad_token_id, mock_labels)
# mock_ids = first_batch["input_ids"].clone().detach()
for batch in wrapper.data_module.val_dataloader():
    print(batch["input_ids"].shape)
    print(batch["labels"].shape)

In [ ]:
wrapper.init_trainer()

In [ ]:
wrapper.tokenizer

In [ ]:
wrapper.train()